# Notebook to match VASCA sources to SIMBAD database sources positionally
Set your option in the cell bellow

In [1]:
from astropy import units as uu
region_name = "TDS" # _ELAISN1
simbad_query = False
radius = 5 * uu.arcsec

In [2]:
%matplotlib widget
import ipywidgets as widgets
from IPython.display import display
import matplotlib.pyplot as plt
from regions import CircleSkyRegion
from astropy.coordinates import SkyCoord
from astropy.wcs.utils import pixel_to_skycoord
from astropy.table import unique, Table
import numpy as np
from vasca.region import Region
from vasca.field import BaseField
import vasca.visualization as vvis
from vasca.tables import TableCollection, dd_vasca_columns
from vasca.utils import dd_ogrp
from astroquery.simbad import Simbad
import warnings
from astropy.utils.exceptions import AstropyWarning
warnings.simplefilter('ignore', category=AstropyWarning)

#prepare file names
region_fname = "./vasca_pipeline/"+region_name+"/region_"+region_name+".fits"
simbad_fname = "./vasca_pipeline/"+region_name+"/simbad_"+region_name+".fits"

#Simbad.get_field_description('otype')

### Query the data or read from file

In [9]:
#Load region
mt = TableCollection()
if simbad_query:
    # Get region and source coordinates
    rg = Region()
    rg.load_from_fits(region_fname)
    sel_src = rg.tt_sources["sel"]
    tt_src = rg.tt_sources[sel_src]
    src_coord = SkyCoord(tt_src["ra"].quantity,tt_src["dec"].quantity,frame='icrs')

    # Run query
    customSimbad = Simbad()
    customSimbad.add_votable_fields('otype',"otype(opt)","otypes")
    tt_simbad = customSimbad.query_region(src_coord,radius=radius)
    
    # Change type to be handabel by astropy
    tt_simbad["MAIN_ID"] = tt_simbad["MAIN_ID"].data.astype("S32")
    tt_simbad["COO_BIBCODE"] = tt_simbad["COO_BIBCODE"].data.astype("S32")
    tt_simbad["OTYPE"] = tt_simbad["OTYPE"].data.astype("S32")
    tt_simbad["OTYPE_opt"] = tt_simbad["OTYPE_opt"].data.astype("S32")
    tt_simbad["OTYPES"] = tt_simbad["OTYPES"].data.astype("S32")

    # Write simbad and source info into matching tables
    src_idx = tt_simbad["SCRIPT_NUMBER_ID"]-1
    tt_simbad["rg_src_id"] = tt_src[src_idx]["rg_src_id"]
    tu_simbad = unique(tt_simbad, keys='SCRIPT_NUMBER_ID')
    tu_simbad["match_id"] = np.array(range(0,len(tu_simbad)), dtype= np.int32)
    display(tu_simbad)
    
    tt_match = Table(tt_src["rg_src_id","ra","dec","pos_err","obs_filter_id","sel","flux","flux_err","flux_nxv"])
    #tt_match["otype"] = OTYPE_opt
    tt_match.meta["rad"]=str(radius)

    # Add tables to table collection and write
    mt.add_table(tt_match,"tt_match")
    mt.add_table(tu_simbad,"tt_simbad")
    mt.add_table(tt_simbad,"tt_simbad_all")

    #Copy information into tt_match table
    otype = np.array([dd_vasca_columns["otype"]["default"]]*len(tt_match), dtype="S32")
    mt.add_column("tt_match","otype",otype)
    mt.tt_match.add_index("rg_src_id")
    mt_idx = mt.tt_match.loc_indices["rg_src_id", mt.tt_simbad["rg_src_id"]]
    mt.tt_match["otype"][mt_idx] = mt.tt_simbad["OTYPE_opt"]
    
    #Write collectiion
    mt.write_to_fits(simbad_fname)
else:
    #Load collection
    mt.load_from_fits(simbad_fname)    
    
# Add object group ID
def add_ogrp_id(tc,tt_name):
    "Helper funtion to add ogrp:id column to tables"
    tt = tc.__dict__[tt_name]
    ogrp_id = np.array([dd_vasca_columns["ogrp_id"]["default"]]*len(tt), dtype="int32")
    tc.add_column(tt_name,"ogrp_id",ogrp_id)
    for ii in range(len(tt)):
        for ogrp_key, ogrp_val in dd_ogrp.items():
            if tt["otype"][ii] in ogrp_val["otypes"]:
                tt["ogrp_id"][ii] = ogrp_val["ogrp_id"]
add_ogrp_id(mt,"tt_match")
display(mt.tt_match)  

<Table length=1030>
rg_src_id         ra                 dec          ...  otype  ogrp_id
                 deg                 deg          ...                
  int32        float64             float64        ... bytes32  int32 
--------- ------------------ -------------------- ... ------- -------
        8 334.64049826925117   0.4526866937010489 ...     QSO       1
       13 334.60403128211425  0.40736589666436374 ...     QSO       1
       34  334.3024804041402 -0.41734493303878567 ...     QSO       1
       75  242.2402236630283    54.35604998949267 ...     QSO       1
       80  242.1485958581278     54.3915070495624 ...     QSO       1
       81 242.13060666557723    54.35237207207334 ...    none      -1
      ...                ...                  ... ...     ...     ...
   122037  34.84828453101386   -4.227723250967488 ...     QSO       1
   122049  34.84703147150364   -4.863533625826487 ...     QSO       1
   122270 34.802767160951646   -4.186673358672989 ...     QSO       1
   122374 34.780172204011116   -3.742706734404377 ...     QSO       1
   122563    34.733191554561   -4.201103112037166 ...     QSO       1
   122861  34.60413135698226    -4.05991213721771 ...       G       2

### Plot found object types

In [12]:
otypes, otype_cts= np.unique(mt.tt_match["otype"],return_counts=True)
fig_otypes = plt.figure("Source types",figsize=(21, 7))
plt.bar( range(len(otypes)), otype_cts, align='center')
plt.xticks(range(len(otypes)), otypes, size='small')
plt.yscale('log')
plt.grid()
plt.xlabel("otype")
plt.ylabel("Nr of sources")
plt.show()

tt_nodes = Table.read('./resources/SIMBAD_otypes/otypes_nodes.csv') #, format='ascii.daophot'
ids, ids_idx, _ = np.intersect1d(tt_nodes["Id"],otypes,return_indices=True)
candidate = np.asarray(np.ma.masked_array(data = tt_nodes["Candidate"],mask = False, fill_value = "none"))
can, can_idx, _ = np.intersect1d(candidate,otypes,return_indices=True)
all_idx = np.unique(np.append(ids_idx,can_idx))
tt_nodes.rename_column('Id', 'otype')
mt.add_table(tt_nodes[all_idx],"tt_otypes")
add_ogrp_id(mt,"tt_otypes")
mt.tt_otypes.pprint_all()#show_dtype=True

otype    Label     Candidate         Description                            Category                                          Subcategory                                            Comment                   Status Priority ogrp_id
                                                                                                                                                                                                                                      
----- ------------ --------- ---------------------------- ------------------------------------------- ------------------------------------------------------------ ------------------------------------------- ------ -------- -------
    ?      Unknown        --     Object of Unknown Nature 8. BLENDS, ERRORS, NOT WELL DEFINED OBJECTS                                                           --                                          --     --       --      -1
    *         Star        --                         Star                   

### Print all source Ids for a given object type

In [8]:
sel_otype = mt.tt_match["otype"] ==  "LI?" # "SN*" #"EB*"
display(mt.tt_match[sel_otype])
print("srcs_ids = [", end="")
for src_id in mt.tt_match[sel_otype]["rg_src_id"].data:
    print(src_id,",", end="")
print("]")

<Table length=1>
rg_src_id         ra        ...           flux_nxv           otype 
                 deg        ...                                    
  int32        float64      ...          float32[2]         bytes32
--------- ----------------- ... --------------------------- -------
     3227 332.7957627981983 ... 0.008711549 .. -0.031486675     LI?

srcs_ids = [3227 ,]
